In [1]:
import os
import pandas as pd
import json
import folium

In [2]:
topojson_europe_path = 'topojson/europe.topojson.json'
topojson_europe = json.load(open(topojson_europe_path))

To improve map interaction, we want to add markers one each country, so that we can display the exact data showed on the map. To do so, we have downloaded a csv of all the countries with the position of their respective capital. We check that the countries in the topojson match the countries in the csv, and we rename the countries that mismatch.
Also, we remove the 'United States' row as it cause a shift in the csv, and we do not need it.
We also remove Vatican city because we don't have data for this country, and it hides Italy data on the map.

In [3]:
# csv for capital locations preprocessing
#http://techslides.com/list-of-countries-and-capitals
countries = pd.read_csv('./europe_data/country-capitals.csv')
countries.index = countries['CountryName']
for i in range(len(topojson_europe['objects']['europe']['geometries'])):
    name = topojson_europe['objects']['europe']['geometries'][i]['properties']['NAME']
    if not name in countries.index:
        print(name)

countries = countries[countries['CapitalLatitude'] != ' D.C.'].rename({'Moldova':'Republic of Moldova',
                                                                      'Macedonia': 'The former Yugoslav Republic of Macedonia'})

The former Yugoslav Republic of Macedonia
Republic of Moldova
Holy See (Vatican City)


In [4]:
def opacity_nan(df, country,year):
    if not country in df[year].dropna().index:
        return 1
    else:
        return 0.0000000001
    
def get_value_or_NaN(df, country, year):
    if country in df.index:
        return str(df.loc[country][year]) + '%'
    else:
        return 'Unavailable data'
    
def generate_map(excel_path, title, legend_title, year, markers=False):
    raw_excel = pd.read_excel(excel_path).dropna()
    df = raw_excel.set_index(raw_excel[title].values).drop(title,axis=1)
    df.columns = df.iloc[0]
    df = df.drop('geo\\time',axis=0).replace(':','NaN').astype(float).rename({
                          'Former Yugoslav Republic of Macedonia, the' :'The former Yugoslav Republic of Macedonia'})
    
    m_europe = folium.Map([60,30],zoom_start=3.4,min_zoom=3.)
    
    m_europe.choropleth(
        geo_data= topojson_europe,
        topojson = 'objects.europe',
        name= legend_title,
        data = df,
        columns=[df.index, year],
        key_on='feature.properties.NAME',
        fill_color='GnBu',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name= legend_title +' (%)'
    )
    
    folium.TopoJson(
    topojson_europe,
    'objects.europe',
    style_function=lambda feature: {
        #not feature['properties']['NAME'] in df_total_unemployment.index:
        'fillColor':  '#5b5858',
        'fillOpacity': opacity_nan(df, feature['properties']['NAME'],year),
        }
    ).add_to(m_europe)
    
    folium.LayerControl().add_to(m_europe)
    
    if markers:
        for i in range(len(topojson_europe['objects']['europe']['geometries'])):
            name = topojson_europe['objects']['europe']['geometries'][i]['properties']['NAME']
            if name in countries.index:
                folium.Marker([countries['CapitalLatitude'].astype(float).loc[name], countries['CapitalLongitude'].astype(float).loc[name]],
                      popup=name + ': ' + get_value_or_NaN(df, name, year),
                      icon=folium.Icon(icon='cloud')
                      ).add_to(m_europe)
    
    return m_europe

In [5]:
def query_map(query_name, sex, year, markers=0):
    year = str(year) #allows to pass year as int argument
    if query_name == 'Unemployment':
        f = open('./europe_data/unemployment_description.txt', 'r')
        print(f.read())
        f.close()
        return generate_map('./europe_data/unemployment_15_74_'+ sex +'.xls',title='Unemployment rate by sex',
             legend_title=sex+' Unemployment in ' + year, year=year, markers=markers)
        
    if query_name == 'Young people Inactivity':
        f = open('./europe_data/no_activity_15_24_people_description.txt', 'r')
        print(f.read())
        f.close()
        return generate_map('./europe_data/no_activity_15_24_people_'+ sex +'.xls',
             title='Young people neither in employment nor in education and training by sex',
             legend_title=sex + ' Young people Inactivity in ' + year, year=year, markers=markers)
    
    

In [10]:
from IPython.html.widgets import interact



html_interact = interact(query_map, query_name=['Unemployment','Young people Inactivity'], sex=['Total','Male','Female'],
        year=range(2005,2017),markers=[0,1])